In [126]:
import sys
sys.path.insert(1, r"C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis")
from Stage1.GAT.GATModel import GraphAttentionNetwork
from Single_Website_Download.Download import main
import torch
from Stage1.ExtractingGraphs.HTMLtoGraph import html_to_graph, EdgeFeatures
from Stage1.ExtractingGraphs.seleniumDriver import driver_init, get_Driver
from Stage1.ExtractingLabels.swde_label_extraction import label_extraction, _find_matches
from Stage1.tree_helpers import *
from pathlib import Path
from scipy import sparse
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [127]:
#Import model
model = GraphAttentionNetwork(in_dim = 114, edge_in_dim = 200, edge_emb_dim = 32, hidden1 = 32, hidden2 = 32, hidden3 = 8, heads = 2)
state_dict = torch.load("../../Stage1/GAT/FULLTRAINEDALLDATAModelf1-75-learning.pt", map_location=torch.device(device))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [128]:
url = r"C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis/data/swde/sourceCode/sourceCode/movie/movie/movie-allmovie(2000)/0000.htm"
#htmlFile = Path(r"C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis/data/websites/test.html")
#main(url,1,htmlFile)
driver_init(True)
A, X, E, edge_index, bbox = html_to_graph(url, get_Driver())#html_to_graph(htmlFile, get_Driver())
A = sparse.csr_matrix(A)
X = sparse.csr_matrix(X)
E = sparse.csr_matrix(E)

In [129]:
def get_random_candidate_edges(filepath):
    TXTLENGTH = 400
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)
    txtNodes = []
    coords = []

    stack = [tree.getroot()]
    while stack:
        #limit to nodes with text less than a certain length. once that node has been read, do not grab its children
        node = stack.pop()
        txtlen = len(get_node_text(node))

        if txtlen > 2 and txtlen < TXTLENGTH:
            txtNodes.append(node2index[node])
            continue  # skip this node's children entirely

        # Reverse so we process in document order.
        for child in reversed(list(node)):
            stack.append(child)

    #This logic should make an undirected graph
    for i in txtNodes:
        for j in txtNodes:
            if i != j:
                coords.append((i,j))

    XPaths: Dict[etree._Element, str] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
    parentMap, depthMap = build_parent_and_depth_maps(tree)
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

def get_specific_candidate_edges(filepath, txt):
    TXTLENGTH = 400
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)
    parentMap, depthMap = build_parent_and_depth_maps(tree)
    node = _find_matches(tree, txt, depthMap)[0]
    nodeindex = node2index[node]
    
    coords = []

    #This logic should make an undirected graph
    for n, _ in iter_elements_with_direct_text(tree):
        i = node2index[n]
        if i != nodeindex:
            coords.append((i,nodeindex))
            coords.append((nodeindex, i))

    XPaths: Dict[etree._Element, str] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

In [130]:
label_index, label_features = get_specific_candidate_edges(url, "Plot Synopsis")
label_index = np.array(label_index)
label_features = sparse.csr_matrix(label_features)

In [131]:
def _npz_to_csr(csr: sparse.csr, dtype=torch.float32):
    crow = torch.from_numpy(csr.indptr.astype(np.int64))
    col  = torch.from_numpy(csr.indices.astype(np.int64))
    val  = torch.from_numpy(csr.data).to(dtype)
    return torch.sparse_csr_tensor(
        crow, col, val, size=csr.shape, dtype=dtype, requires_grad=False
    )

def _npy_to_tensor(arr: np.ndarray, dtype=torch.long):
    return torch.from_numpy(arr).to(dtype).t().contiguous()

X, Aei, Aef, Lei, Lef = _npz_to_csr(X), _npy_to_tensor(edge_index), _npz_to_csr(E), _npy_to_tensor(label_index), _npz_to_csr(label_features)
X, Aei, Aef, Lei, Lef = X.to(device), Aei.to(device), Aef.to(device), Lei.to(device), Lef.to(device)
print(Lef.shape)

torch.Size([350, 200])


In [132]:
model.eval()
logits = model(X, Aei, Aef, Lei, Lef)
probs  = torch.sigmoid(logits)
probs.shape

torch.Size([350])

In [133]:
order = np.argsort(probs.squeeze().tolist())[::-1]
sorted_label_index = label_index[order]
sorted_label_index

array([[302, 301],
       [301, 302],
       [301, 448],
       [301, 280],
       [301, 280],
       [301, 275],
       [301, 275],
       [301, 450],
       [301, 322],
       [301, 273],
       [301, 273],
       [301, 445],
       [301, 316],
       [301, 269],
       [301, 269],
       [301, 310],
       [301, 257],
       [301, 267],
       [301, 267],
       [301, 449],
       [301, 256],
       [301, 312],
       [301, 263],
       [301, 263],
       [301, 277],
       [301, 277],
       [301, 308],
       [301, 320],
       [301, 261],
       [301, 261],
       [301, 444],
       [301, 265],
       [301, 265],
       [301, 314],
       [301, 326],
       [301, 278],
       [301, 306],
       [584, 301],
       [257, 301],
       [301, 259],
       [301, 259],
       [301, 271],
       [301, 271],
       [301,  17],
       [301, 324],
       [301, 584],
       [301, 511],
       [301,  16],
       [301, 318],
       [593, 301],
       [511, 301],
       [301, 325],
       [301,

In [134]:
tree = load_html_as_tree(url)
_, index2node = bfs_index_map(tree)

for label in sorted_label_index:
    print(get_node_text(index2node[label[0]], True).strip(), "->", get_node_text(index2node[label[1]], True).strip())
    print(tree.getpath(index2node[label[0]]), "->", tree.getpath(index2node[label[1]]))

byjasonankeny -> plotsynopsis
/html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[2] -> /html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[1]
plotsynopsis -> byjasonankeny
/html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[1] -> /html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[2]
plotsynopsis -> egoyan
/html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[1] -> /html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[2]/td/p/a[3]
plotsynopsis -> thewarzone1999timroth
/html/body/div/div[3]/table/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div/div/table/tbody/tr[1]/td[1] -> /html/body/div/div[3]/table/tbody/tr/td[1]/t